# 📦 Instalacja pakietu `xnat_dcm2bids`

In [82]:
!pip install --force-reinstall https://github.com/nencki-lobi/xnat_dcm2bids/archive/refs/heads/master.zip > /dev/null 2>&1

# Spis rzeczy:

1. [dcm2bids](https://unfmontreal.github.io/Dcm2Bids/)
    - dcm2bids_scaffold - przygotowuje nowy katalog BIDS  
      `dcm2bids_scaffold -o /home/jovyan/bids-dir`
    - dcm2bids - konwersja i sortowanie zgodnie z BIDS  
      `dcm2bids -c config.json -p 01 -s 01 -o ./ -d ./sourcedata/12345678`
2. [xnat-utils](https://github.com/Australian-Imaging-Service/xnatutils)
    - xnat-get - pobiera wszystkie pliki z XNAT'a w formacie DICOM  
      `xnat-get -p mj_ris -t ./sourcedata`
3. LOBI scripts & wrappers
    - xnat_getcsv - pobiera listę sesji z XNATa do CSV   
      `xnat_getcsv mj_ris.csv mj_ris`
    - xnat_dcm2bids - wrapper dla xnat-get & dcm2bids; pobiera i konwertuje dane z XNAT do BIDS  
      `xnat_dcm2bids --bids-dir ~/bids-dir --config ./code/config.json f490f566-2124-46 03 01`
    - lobi_script - skrót to uruchamiania skryptów z repozytorium [lobi-mri-scripts](https://github.com/nencki-lobi/lobi-mri-scripts)  
      `lobi_script run_mriqc.sh 03 ~/bids-dir ~/bids-dir/derivatives/mriqc`




# 🔐 Konfiguracja połączenia z XNAT

Przejdź do okienka Terminala i wpisz `xnat-get`, żeby skonfigurować połączenie z XNAT.  

Powinno się pojawić `ERROR! project_id ("-p") must be provided to use empty IDs string`

# 📥 Pobranie listy sesji z projektu XNAT

In [83]:
%cd ~
!xnat_getcsv mj_ris.csv mj_ris

/home/jovyan
🟢 Data saved to mj_ris.csv


In [ ]:
import pandas as pd
df = pd.read_csv("mj_ris.csv",dtype=str,index)
df.head(10)

# 🏗️ Przygotowanie struktury katalogów BIDS

In [94]:
bidsdir="/home/jovyan/bids-dir"

In [ ]:
!dcm2bids_scaffold -o {bidsdir}

In [96]:
%cd {bidsdir}

/home/jovyan/bids-dir


In [97]:
!ls -1 {bidsdir}

CHANGES
README
code
dataset_description.json
derivatives
participants.json
participants.tsv
sourcedata
sub-03
tmp_dcm2bids


# Skopiowanie gotowej konfiguracji dla dcm2bids

In [98]:
!ln -sf ~/lobi-mri-scripts/dcm2bids/mjris.json {bidsdir}/code/config.json

# ▶️ Przetworzenie jednej sesji

xnat session id: `f490f566-2124-46`  
subject: `03`  
session: `01`


In [ ]:
!xnat_dcm2bids --bids-dir {bidsdir} --config ./code/config.json f490f566-2124-46 03 01

# 🔁 Przetworzenie wielu sesji z CSV

In [ ]:
!while IFS=',' read -r id _ sub ses _; do \
  echo xnat-dcm2bids --config ./code/config.json "$id" "$sub" 1; \
done < ~/mj_ris.csv

# 🔁 Uruchomienie skryptów mriqc, fmriprep etc.

Uwaga! Wymagane jest środowisko neurodesk (Calcus, Mikołajki ew. neurodesktop).

In [ ]:
!lobi_script run_mriqc.sh 03 {bidsdir} {bidsdir}/derivatives/mriqc

In [ ]:
!tail -n +2 ~/mj_ris.csv | parallel --colsep ',' echo lobi_script run_fmriprep_min.sh {{3}} {bidsdir} {bidsdir}/derivatives/fmriprep
